In [20]:
#import dependencies
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
#import api key from file
from apikey import googlekey

# Configure gmaps
gmaps.configure(api_key=googlekey)

In [21]:
!jupyter nbextension enable --py --sys-prefix widgetsnbextension
!pip install gmaps
!jupyter nbextension enable --py --sys-prefix gmaps



Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: ok


Enabling notebook extension jupyter-gmaps/extension...
      - Validating: ok


In [22]:
csvpath = os.path.join("..","Data","forecast_data.csv")

# read in WeatherPy CSV file
citydata = pd.read_csv(csvpath)
display(citydata)

,Unnamed: 0,City Name,Latitude,Longitude,Max Temperature,Humidity,Cloudiness,Wind Speed
0,0,Qaanaaq,77.4840,-69.3632,32.11,77,97,11.68
1,1,Qasigiannguit,68.8193,-51.1922,34.70,81,40,5.68
2,2,Baruun-Urt,46.6806,113.2792,43.74,20,12,15.57
3,3,Bluff,-46.6000,168.3333,57.61,80,98,23.62
4,4,Ushuaia,-54.8000,-68.3000,43.23,82,96,11.65
...,...,...,...,...,...,...,...,...
554,554,Atar,20.5169,-13.0499,100.13,17,4,13.94
555,555,Ban Dung,17.6990,103.2596,81.64,90,100,13.09
556,556,Santiago de Cuba,20.0247,-75.8219,89.60,41,7,10.42
557,557,Payo,-3.7508,103.6372,92.39,50,66,3.00


In [23]:
citydata.Latitude.max()

78.2186

In [24]:
# plot heatmap of city humidity on google maps
citylocs = citydata[["Latitude","Longitude"]]
humidity = citydata["Humidity"]

fig = gmaps.figure()

humidity_layer = gmaps.heatmap_layer(citylocs,weights=humidity,
                                     dissipating=False,max_intensity =100,
                                     point_radius=1)
fig.add_layer(humidity_layer)
display(fig)

Figure(layout=FigureLayout(height='420px'))

In [19]:
# filter city data to find cities with ideal weather conditions
idealvacation = citydata[(citydata["Humidity"] < 40)
                         & (citydata["Max Temperature"]< 85)
                         & (citydata["Max Temperature"]> 60)
                         & (citydata["Cloudiness"] < 50)
                         & (citydata["Wind Speed"]>10)
                         & (citydata["Wind Speed"]<20)]
display(idealvacation)

,Unnamed: 0,City Name,Latitude,Longitude,Max Temperature,Humidity,Cloudiness,Wind Speed
51,51,Lompoc,34.6391,-120.4579,71.83,39,0,14.50
61,61,Bījār,35.8668,47.6051,71.02,21,0,12.46
68,68,Alice Springs,-23.7000,133.8833,75.99,30,0,14.29
126,126,Yulara,-25.2406,130.9889,78.37,27,0,13.00
292,292,Seoul,37.5683,126.9778,70.12,23,23,14.27
314,314,Gobabis,-22.4500,18.9667,82.04,24,0,16.51
318,318,Faya,18.3851,42.4509,69.93,38,11,13.38
330,330,San Rafael,-34.6177,-68.3301,75.56,35,0,17.29
335,335,Abhā,18.2164,42.5053,69.15,39,14,13.20
369,369,Saint George,37.1041,-113.5841,76.06,25,0,15.61


In [ ]:
#for each city in citylist
    # Call google places API to find the first hotel result within 5000 meters of each city
    #save hotel name, city, and country data to an existing list or dictionary


In [ ]:
#plot these hotels on the humidity heatmap with pins containing hotel name, city, and country